# Lightweight Quantum: BFCD with Bilinear Entanglement (High Reps)

This notebook implements the **Lightweight Quantum** approach for portfolio optimization:
- **Ansatz**: BFCD with bilinear entanglement
- **Parameters**: High reps (3-4) for deep circuit exploration
- **Expected**: Fastest to target quality → Fewer parameters, efficient gates, deep exploration

## Setup and Imports

In [1]:
import os
import sys
import time
import numpy as np
import pickle
from pathlib import Path
from dataclasses import asdict

# add root directory to path
ROOT = Path(os.getcwd()).parent
sys.path.append(str(ROOT))

# quantum imports
from qiskit_aer import AerSimulator

# project imports
# 1. LP/QUBO processing
from src.sbo.src.utils.lp_utils import load_quadratic_program
from src.sbo.src._converters.quadratic_program_to_qubo import QuadraticProgramToQubo
# 2. ansatz building
from src.step_1 import build_ansatz
# 3. quantum optimization
from src.sbo.src.patterns.building_blocks.step_3 import HardwareExecutor
# 4. experiment saving
from src.experiment import Experiment

print(f"Working directory: {ROOT}")
print(f"Python path includes: {str(ROOT)} in sys.path")

Working directory: C:\Users\ASUS\Downloads\WISER_Optimization_VG-main
Python path includes: C:\Users\ASUS\Downloads\WISER_Optimization_VG-main in sys.path


## Problem Setup: 31-Bond Portfolio Optimization

The problem starts as an LP file, which is converted into a quadratic function and then transformed into a Quadratic Unconstrained Binary Optimization (QUBO) problem using predefined modules.

In [2]:
# load the 31-bond problem
lp_file_path = str(ROOT) + '/data/1/31bonds/docplex-bin-avgonly-nocplexvars.lp'

# convert LP file into quadratic problem
quadratic_problem = load_quadratic_program(lp_file_path)

# convert quadratic program into QUBO
converter = QuadraticProgramToQubo()
qubo = converter.convert(quadratic_problem)

print(f"31-bond problem: {quadratic_problem.get_num_binary_vars()} binary variables")
print(f"QUBO variables: {qubo.get_num_vars()}")
print(f"QUBO objective sense: {qubo.objective.sense}")
print(f"QUBO constant term: {qubo.objective.constant}")

31-bond problem: 31 binary variables
QUBO variables: 31
QUBO objective sense: ObjSense.MINIMIZE
QUBO constant term: 0.0


## Objective Function

The objective function is given by the formula:

$$
\min \sum_{\ell \in L} \sum_{j \in J} \rho_j \left( \sum_{c \in K_\ell} \beta_{c,j} \, x_c - K_{\ell,j}^{\text{target}} \right)^2
$$

The goal is to minimize the difference between a portfolio's characteristics and their target values, ensuring the portfolio matches the targets as closely as possible. This objective function is handled internally by the QUBO formulation.

In [3]:
# create objective function
def objective_function(x):
    """Objective function for the QUBO problem"""
    return qubo.objective.evaluate(x)

## Backend Simulator

**AerSimulator** is a concrete implementation of **BackendV2**, which is the abstract interface/protocol for all quantum backends in Qiskit.

In [4]:
# create backend simulator (AerSimulator)
num_vars = qubo.get_num_vars()
aer_options = {'method': 'matrix_product_state', 'n_qubits': num_vars}
backend = AerSimulator(**aer_options)

print(f"Backend: {backend}")
print(f"Number of qubits: {num_vars}")

Backend: AerSimulator('aer_simulator_matrix_product_state')
Number of qubits: 31


## Ansatz Configuration

In [5]:
# BFCD configuration
ansatz = 'bfcd'
ansatz_params = {
    'reps': 3,  # high reps for deep circuit exploration
    'entanglement': 'bilinear',  # BFCD default (most efficient)
}

print(f"Configuration: {ansatz} with {ansatz_params}")
print(f"Expected: Parameter-efficient with deep circuits")

Configuration: bfcd with {'reps': 3, 'entanglement': 'bilinear'}
Expected: Parameter-efficient with deep circuits


## Ansatz Building

In [6]:
# build BFCD ansatz
ansatz_circuit, initial_layout = build_ansatz(
    ansatz=ansatz,
    ansatz_params=ansatz_params,
    num_qubits=num_vars,
    backend=backend
)

print(f"BFCD Ansatz has {ansatz_circuit.num_parameters} parameters")
print(f"Circuit depth: {ansatz_circuit.depth()}")
print(f"Number of gates: {len(ansatz_circuit.data)}")
print(f"Entanglement: {ansatz_params['entanglement']}")
print(f"Repetitions: {ansatz_params['reps']}")

BFCD Ansatz has 273 parameters
Circuit depth: 34
Number of gates: 665
Entanglement: bilinear
Repetitions: 3


## Conditional Value at Risk (CVaR)

CVaR is a generalization of both the sample mean and the minimum. It represents the expected value of the lowest α-fraction (i.e., the lower tail) of a distribution. In this case, the distribution of sampled energy values from a quantum Hamiltonian.

According to Barkoutsos et al. (2020), to compute CVaR:

- The Hamiltonian is sampled (measured indirectly) *K* times.
- Each sample yields an energy value.
- The energy values are sorted in ascending order (lowest to highest).
- CVaR is then calculated as the average of the lowest ⌈αK⌉ values using the formula:

$$
\text{CVaR}_\alpha = \frac{1}{\lceil \alpha K \rceil} \sum_{k=1}^{\lceil \alpha K \rceil} H_k
$$

Here, $H_k$ denotes the k-th lowest observed energy value.

In implementation:

- The CVaR calculation is performed by the function <code>calc_cvar()</code> in <code>"src/sbo/src/patterns/building_blocks/step3.py"</code>.
- Sampled measurement results are mapped to energy values by the function <code>_sampler_result_to_cvar()</code> in <code>"src/sbo/src/patterns/building_blocks/step3.py"</code>.

## Standard VQE

After translating a problem into QUBO form and constructing a Hamiltonian for an n-qubit system, the standard VQE algorithm samples (i.e., indirectly measures) the Hamiltonian *K* times. These measurement outcomes are summed and averaged to estimate the expectation value $\langle \psi(\theta) | H | \psi(\theta) \rangle$. The lowest expectation value, corresponding to a specific set of parameters in the chosen ansatz that minimizes the cost function, is taken as the optimal solution (bitstring) with high probability.

## CVaR-VQE

In contrast, CVaR-VQE optimizes a CVaR objective, focusing on the average of the worst $\alpha$-fraction of sampled energies. The minimal CVaR value, corresponding to certain ansatz parameters minimizing this objective, is considered the optimal solution (bitstring).

## Hardware Executor

The HardwareExecutor orchestrates the entire VQA process. It initializes quantum circuit parameters using π/3 (≈1.047) as a reference value, which provides a good starting point for optimization. The CVaR parameters are set to focus on the worst 10% of measurements (alpha=0.1) with 1024 quantum measurements per iteration.

The executor uses the NFT (Natural Frequency Tuning) optimizer with the following configuration:
- **objective_fun**: QUBO objective function to minimize
- **backend**: Quantum simulator backend
- **isa_ansatz**: BFCD quantum circuit ansatz
- **optimizer_theta0**: Initial parameter values
- **optimizer_method**: NFT optimizer
- **sampler_options**: Quantum measurement settings
- **solver_options**: Maximum 10 iterations, CVaR parameter, random parameter updates

In [7]:
# initialize quantum circuit parameters
theta_initial = np.pi/3 * np.ones(ansatz_circuit.num_parameters)

# cvar optimization parameters
alpha = 0.1   # cvar parameter: focus on worst 10% of measurements
shots = 1024  # number of quantum measurements per iteration

# create HardwareExecutor for quantum optimization
he = HardwareExecutor(
    objective_fun=objective_function,
    backend=backend,
    isa_ansatz=ansatz_circuit,
    optimizer_theta0=theta_initial,
    optimizer_method="nft",
    refvalue=None,
    sampler_options={'default_shots': shots},
    use_session=False,
    verbose="progress",
    file_name=None,
    store_all_x=True,
    solver_options={
        "max_epoch": 10,
        "alpha": alpha,
        "random_update": True
    }
)

## Simulation

The simulation executes the BFCD VQA optimization with CVaR. The process involves:

1. Initialization: Quantum circuit parameters are set using π/3 as reference values
2. Iteration Loop: For each optimization iteration:
   - Prepare quantum state using BFCD ansatz
   - Measure Hamiltonian expectation value K times (shots)
   - Calculate CVaR from the worst α-fraction of measurements
   - Update parameters using NFT optimizer
3. Convergence: Continue until convergence or max_epoch limit

The optimization extracts the best solution found, which includes:
- **best_x**: Binary vector representing selected bonds (1=selected, 0=not selected)
- **best_fx**: Objective function value (CVaR) of the best solution

In [8]:
# run the bfcd VQA optimization with cvar
print("Starting BFCD VQA optimization with CVaR...")

# record start time for performance measurement
start_time = time.time()

# execute the variational quantum optimization
result = he.run()

# record end time and calculate total optimization time
end_time = time.time()
print(f"Optimization completed in {end_time - start_time:.2f} seconds")

# extract the best solution found during optimization
best_x = he.optimization_monitor.objective_monitor.best_x
best_fx = he.optimization_monitor.objective_monitor.best_fx

print(f"Best solution: {best_x}")
print(f"Best objective value: {best_fx}")

2025-10-02 12:30:31,652 INFO optimization_wrapper: run...
2025-10-02 12:30:31,660 DEBUG optimization_wrapper: Using X0 [1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.0471975

Starting BFCD VQA optimization with CVaR...


2025-10-02 12:30:31,664 INFO nft: Using parameters: random_update True
2025-10-02 12:30:31,668 INFO nft: Using parameters: epoch_start 0
2025-10-02 12:30:31,670 INFO nft: Using parameters: idx_set_start None
2025-10-02 12:30:31,673 INFO nft: Using parameters: step_start 0
2025-10-02 12:30:35,114 INFO optimization_monitor: optimizer internal status: () {'idx_set': array([ 95, 241,  13, 217, 113, 139, 159, 225, 122,  52, 229, 265,  38,
       136, 204,  84,  17, 245, 259, 261, 131, 133,  57,  18, 121,   0,
       104, 215, 108, 203,   3, 257, 171, 205, 142,  19, 260,   2, 148,
       243, 116, 181, 237, 157, 233, 182, 161,  41,  53, 230, 249, 126,
        16,   9,   8, 125,  63, 185,  54,  75, 232,  83, 167, 240,  99,
       162, 134, 210, 138,  25,  22, 169,  48, 235, 137,  15, 263, 270,
       220, 180,   6, 234, 114, 168,  94,  20,  23, 198, 193,  56, 188,
       258, 255,  93, 214, 147, 262,  39, 242,  44, 151,  85, 107, 199,
        82,  43,   7, 246,  51, 143, 272, 112,   1,  76, 2

Optimization completed in 11703.83 seconds
Best solution: [1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 0. 1.]
Best objective value: -2237.1363660885936


## Result

The experiment is saved as a pickle file containing all the necessary data for analysis, reproducibility, debugging, and research. The saved data includes optimization results, convergence data, performance metrics, configuration settings, and experiment metadata.

The experiment object includes:
- Configuration: Ansatz, parameters, optimizer settings, CVaR alpha
- Performance: Execution time, iterations, convergence behavior
- Results: Best solution, objective value, relative gap
- Monitoring: Parameter evolution, objective function calls, optimization trajectory
- Hardware: Backend, shots, classical hardware specs

In [9]:
# set path
experiment_path = Path(str(ROOT)+"/project/results/exp_lightweight_quantum.pkl")

In [10]:
# create an Experiment object from the results
experiment = Experiment.from_step3(
    experiment_id="lightweight_bfcd_run",
    ansatz="bfcd",
    ansatz_params={'reps': 3, 'entanglement': 'bilinear'},
    theta_initial="piby3",
    device="AerSimulator",
    optimizer="nft",
    alpha=alpha,
    theta_threshold=None,
    lp_file="31bonds/docplex-bin-avgonly-nocplexvars.lp",
    shots=shots,
    refx=None,      # set this for a reference solution
    refvalue=None,  # set this for a reference value
    classical_hw=Experiment.get_current_classical_hw(),
    step3_time=end_time - start_time,
    step3_job_ids=he.job_ids,
    result=result,
    optimization_monitor=he.optimization_monitor
)

# save the experiment for analysis
experiment_path.parent.mkdir(parents=True, exist_ok=True)
with open(experiment_path, 'wb') as f:
    pickle.dump(asdict(experiment), f)